In [3]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

print("Sdk version is:", azureml.core.VERSION)

Sdk version is: 1.19.0


In [4]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.subscription_id, ws.location, sep= "\n")

quick-starts-ws-135474
aml-quickstarts-135474
9b72f9e6-56c5-4c16-991b-19c652994860
southcentralus


In [5]:
experiment_name = "automl-exp"
project_folder = "./automl-project"

exp = Experiment(ws, experiment_name)
exp

Name,Workspace,Report Page,Docs Page
automl-exp,quick-starts-ws-135474,Link to Azure Machine Learning studio,Link to Documentation


In [7]:
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

cluster_name= "AML-Cluster"

#Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
    
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_v2',
                                                           min_nodes=1, max_nodes=5)
    
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
compute_target.get_status()

Creating
Succeeded.....................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
#load the data
found = False
key = "Bank-marketing"
description_text = "Bank-Marketing Dataset for Second Project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
#view the data
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [9]:
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.040212,257.335205,2.561730,962.174780,0.174780,0.076228,93.574243,-40.518680,3.615654,5166.859608
std,10.432313,257.331700,2.763646,187.646785,0.496503,1.572242,0.578636,4.623004,1.735748,72.208448
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,318.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [10]:
#automl experiment settings and configuration
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             enable_onnx_compatible_models=True,
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [11]:
#submit the experiment
automl_run = exp.submit(automl_config, show_output= True)

Running on remote.
No run_configuration provided, running on AML-Cluster with default configuration
Running on remote compute: AML-Cluster
Parent Run ID: AutoML_c501ddd8-cd63-48c0-97b8-515c8330f6b2

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Train-Test data split
STATUS:       DONE
DESCRIPTION:  Your input data has been split into a training dataset and a holdout test dataset for validation of the model. The test holdout dataset reflects the original distribution of your input data.
              
DETAILS:      
+---------------------------------+---------------------------------+-------------------------

In [12]:
#Retrieve and save onnx model
from azureml.automl.runtime.onnx_convert import OnnxConverter

best_model, onnx_model= automl_run.get_output(return_onnx_model=True)
onnx_path = "./best_automl_model.onnx"
OnnxConverter.save_onnx_model(onnx_model, file_path=onnx_path)